In [ ]:
open System.Net.Http
open System.IO
open System.Net.Http.Headers
open System.Net

let getWebString (url: string) =
    async {
        use client = new HttpClient(
            new HttpClientHandler(
                AutomaticDecompression = DecompressionMethods.All
            ))
        client.DefaultRequestHeaders.Accept.Add(new MediaTypeWithQualityHeaderValue("text/html"))

        let! response = 
            client.GetAsync(url)
            |> Async.AwaitTask
        let! content = 
            response.Content.ReadAsStringAsync()
            |> Async.AwaitTask
        return content
    }

open System.Text.RegularExpressions
let extractWords (page: string) =
    let rex = new Regex("<span class=mot[2]?>(.*?)</span>")
    let matches = rex.Matches(page)
    let strings =
        matches
        |> Seq.map (fun mc -> mc.Groups[1].Value)
        |> List.ofSeq
    (String.concat "" strings).Split(" ") |> List.ofArray

let content = 
    getWebString "https://www.bestwordlist.com/5letterwords.htm"
//    getWebString "https://www.vg.no"
    |> Async.RunSynchronously

let getWordsForSite url = 
    async {
        let! content = getWebString url
        return extractWords content
    }

let urls = 
    [for i in 1..15 do if i = 1 then yield "" else yield sprintf "page%i" i]
    |> List.map (fun i -> sprintf "https://www.bestwordlist.com/5letterwords%s.htm" i)

let words =
    urls
    |> List.map (fun url -> getWordsForSite url)
    |> List.map (fun task -> Async.RunSynchronously task)
    |> List.ofSeq
    |> List.concat
    |> List.filter (fun w -> w.Length = 5)
    |> List.distinct


In [ ]:
// TRAMS
let fixedLetters = [0, 'A'; 3, 'E'] |> Map.ofList
//let fixedLetters = [2, 'A'] |> Map.ofList

let excludedCharacters = "TRMSLCIDWOKNX"
let includeLetters = [('A', [2]);'E',[4]] |> List.map (fun (x,y) -> x,(y |> Set.ofList))

let filterCandidates (wordList: string list) (fixedLetters: Map<int, char>) (excludedCharacters: string) (includeLetters: (char*(Set<int>)) list) =
    let excludedCharactersArr = excludedCharacters.ToCharArray()
    let result = 
        wordList
        |> List.filter (fun w -> 
            let chars = w.ToCharArray()
            let letterComparison = 
                [ for i in 0 .. 4 do
                    let currentChar = chars[i]
                    if excludedCharactersArr |> Array.contains currentChar then yield false
                    else if fixedLetters |> Map.containsKey i && fixedLetters[i] <> currentChar then yield false
                    else yield true
                ] |> List.contains false |> not
            let wordIndicesLookup = chars |> Array.mapi (fun x c -> c,x) |> Array.groupBy fst |> Map.ofArray |> Map.map (fun _ grp -> grp |> Array.map snd |> Set.ofArray)
            let includeComparison =
                includeLetters
                |> List.map (fun (letter, indices) ->
                    let letterIndices = wordIndicesLookup |> Map.tryFind letter |> Option.defaultValue Set.empty
                    let diff = Set.difference letterIndices indices
                    diff |> Set.isEmpty |> not
                ) |> List.contains false |> not
            letterComparison && includeComparison
        )
    result

try
    filterCandidates words fixedLetters excludedCharacters includeLetters
    |> printfn "==> Filtered candidates: %A"
with
| e -> printfn "==> Error: %A" e 

==> Filtered candidates: ["ABBEY"]
